In [1]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
"""sudo ./GIZA++ -S ../corpus.rup.vcb -T ../corpus.ro.vcb -C ../corpus.rup_corpus.ro.snt -CoocurrenceFile ../corp.cooc -o Result -outputhpath ../test"""

In [2]:
alignment_dict = dict()
#Alinieri cu eflomal
def read_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file]

def parse_alignments(alignment_file, source_sentences, target_sentences):
    with open(alignment_file, 'r', encoding='utf-8') as file:
        for line_num, line in enumerate(file):
            alignment_pairs = line.strip().split()
            for pair in alignment_pairs:
                src_index, tgt_index = map(int, pair.split('-'))
                src_word = source_sentences[line_num].split()[src_index]
                tgt_word = target_sentences[line_num].split()[tgt_index]
                yield src_word, tgt_word

# Read sentences from source and target files
source_sentences = read_sentences('corpus.rup')
target_sentences = read_sentences('corpus.ro')

# Extract and print aligned word pairs
for src_word, tgt_word in parse_alignments('forward.align', source_sentences, target_sentences):
    print(f"{src_word} - {tgt_word}")
    alignment_dict[src_word] = tgt_word

LITERATURĂ - LITERATURĂ
POPULARĂ - POPULARĂ
ANECDOTE - ANECDOTE
LEGENDE - LEGENDE
BASME - BASME
LUPLU - LUPUL
CU - ȘI
VULPEA - VULPEA
Eara - fost
țe - ce
nu - n-a
ș-eara. - fost.
Eara - fost
un - un
lup, - lup,
- - -
Masturlu-Nicola, - Cumătrul-Nicola,
și - și-o
vulpe, - vulpe,
- - -
Chira-Mara. - Cumătra-Mara.
Nîși - doi
s-feațiră - s-au
soț, - întovărășit,
se-avină - vâneze
deadun - împreună,
și - iar
avinaticlu - vânatul,
s-lu-mpartă - și-au
frățește. - zis,
avinaticlu - l-or
s-lu-mpartă - împărți
frățește. - frățește.
Tu - Ieșind
ișirea - vânat,
- - dar,
aflară - peste
ună - o
oală - oală
cu - cu
ñeare. - miere.
- - -
Ha!, - Aha!,
s-aurñi - repezi
Masturlu-Nicola, - Cumătru-Nicola,
- - -
u - o
mîc! - pap!
- - -
Năinte - întâi
ș-năinte - întâi
nu - nu
prinde - trebuie
s-dzîț: - spui:
„va - „o
mîc“, - pap!“,
ma: - ci
„va - „o
mîcăm“, - păpăm“,
îl’i - i-a
fațe - zis
Chira-Mara. - Cumătra-Mara.
Ș-apoia, - Apoi,
nu - nu
dzîț - gândești,
ninga - nici
nu - n-am
și, - oală,
ai! - gata:
s-u

In [25]:
"""Traduceri folosind dictionarul de alinieri. Apoi folosim diferite metrici pentru a evalua traducerile. Datele constau in propozitiile in romana si aromana
Traducem propozitiile din romana in aromana si comparam cu originalul in romana."""
rup_text = read_sentences('annotations/annotations_rup2.txt')
translations = []
for sent in rup_text:
    sent_translation = []
    for word in sent.split():
        if word in alignment_dict:
            sent_translation.append(alignment_dict[word])
        else:
            sent_translation.append(word)
    translations.append([' '.join(sent_translation)])

ro_text = read_sentences('annotations/annotations_ro2.txt')
ro_text = [[sent] for sent in ro_text]

bleu_score = corpus_bleu(ro_text, translations)
print("BLEU Score:", bleu_score)
    

BLEU Score: 0


In [28]:
def jaccard_similarity(sentence1, sentence2):
    words_set1 = set(sentence1.split())
    words_set2 = set(sentence2.split())
    intersection = len(words_set1.intersection(words_set2))
    union = len(words_set1) + len(words_set2) - intersection
    return intersection / union


similarity_total = 0
for sent, i in zip(ro_text, range(len(ro_text))):
    print(sent, translations[i])
    similarity = jaccard_similarity(*sent, *translations[i])
    similarity_total += similarity

mean_similarity = similarity_total / len(rup_text)
print("Mean Jaccard similarity:", mean_similarity)


['Se întuneca de-a binelea.'] ['întuneca binelea.']
['Frumoasa-lumii auzind nenorocirea - că i-au omorât bărbatul'] ['Frumoasa-lumii auzind nișanea ce întâmplat lasă pierit bărbatul,']
['Haida mea, ori mor și eu cu tine, ori fugim împreună sâmbătă noaptea.'] ['Haido, ori mor o mor cu tine, ori împărtășit sâmbătă noapte o Fă-te']
['Din zare se vedeau trei călători, care veneau călare.'] ['de fugă pâlpâind trei călători, care veneau călare.']
['În fruntea lor era un tânăr, mândru și frumos ca un înger.'] ['satului. vreun fruntea un tânăr, mândru frumos din ce se tare un înger.']
['Lumea se risipi, și cei doi bătrâni plecară și ei, încet-încet.'] ['Lumea risipi, doi bătrâni plecară ei, încet-încet.']
['Catârii fură ușurați de poveri, oile se revărsară după iarbă înrourată.'] ['Catârii se poveri, oile revărsară pe în câmpia înrourată.']
['Și cânta Dinu cu atâta foc, cu atâta dor, că până și satul pustiu se îndurera.'] ['iar cânta atâta foc, atâta dor, că satul pustiu s-cutrimură din îndure

In [ ]:
pos_tagged = nltk.pos_tag(prop_nlp)
